# Calculate second lowest cost silver plan (SLCSP)
--- 

## Problem
---
You have been asked to determine the second lowest cost silver plan (SLCSP) for a group of ZIP Codes.


## Task
---
You have been given a CSV file, `slcsp.csv`, which contains the ZIP Codes in the first column. Fill in the second column with the rate (see below) of the corresponding SLCSP. Your answer is the modified CSV file, plus any source code used.

Write your code in your best programming language.

The order of the rows in your answer file must stay the same as how they appeared in the original `slcsp.csv`.

It may not be possible to determine a SLCSP for every ZIP Code given. Check for cases where a definitive answer cannot be found and leave those cells blank in the output CSV (no quotes or zeroes or other text).

## Additional information
---
The SLCSP is the so-called "benchmark" health plan in a particular area. It is used to compute the tax credit that qualifying individuals and families receive on the marketplace. It is the second lowest rate for a silver plan in the rate area.

For example, if a rate area had silver plans with rates of `[197.3, 197.3, 201.1, 305.4, 306.7, 411.24]`, the SLCSP for that rate area would be `201.1`, since it is the second lowest rate in that rate area.

A plan has a "metal level", which can be either Bronze, Silver, Gold, Platinum, or Catastrophic. The metal level is indicative of the level of coverage the plan provides.

A plan has a "rate", which is the amount that a consumer pays as a monthly premium, in dollars.

A plan has a "rate area", which is a geographic region in a state that determines the plan's rate. A rate area is a tuple of a state and a number, for example, NY 1, IL 14.

There are two additional CSV files in this directory besides `slcsp.csv`:

* `plans.csv` -- all the health plans in the U.S. on the marketplace
* `zips.csv` -- a mapping of ZIP Code to county/counties & rate area(s)
A ZIP Code can potentially be in more than one county. If the county can not be determined definitively by the ZIP Code, it may still be possible to determine the rate area for that ZIP Code.

A ZIP Code can also be in more than one rate area. In that case, the answer is ambiguous and should be left blank.

We will want to compile your code from source and run it, so please include the complete instructions for doing so in a COMMENTS file.

In [311]:
import pandas as pd

## Examine data characteristics

In [312]:
def print_first_line(filename):
    name = filename.split('.')[1]
    print('\n{}'.format(name[1:].upper()))
    
    df = pd.read_csv(filename)
    print('\nHeader: \n{}'.format(df.head()))
    print('\nShape: {}'.format(df.shape))
    print('\nDups: {}'.format(sum(df.duplicated())))
    print('\n')
    
data_files = ['./plans.csv',
              './slcsp.csv',
              './zips.csv']

for file in data_files:
    print_first_line(file)


PLANS

Header: 
          plan_id state metal_level    rate  rate_area
0  74449NR9870320    GA      Silver  298.62          7
1  26325VH2723968    FL      Silver  421.43         60
2  09846WB8636633    IL        Gold  361.69          5
3  78590JQ3204809    SC        Gold  379.97          2
4  21163YD2193896    AR        Gold  359.75          2

Shape: (22239, 5)

Dups: 0



SLCSP

Header: 
   zipcode    rate
0    64148  139.91
1    67118  122.42
2    40813  158.12
3    18229  139.91
4    51012  139.91

Shape: (51, 2)

Dups: 0



ZIPS

Header: 
   zipcode state  county_code     name  rate_area
0    36749    AL         1001  Autauga         11
1    36703    AL         1001  Autauga         11
2    36003    AL         1001  Autauga         11
3    36008    AL         1001  Autauga         11
4    36006    AL         1001  Autauga         11

Shape: (51541, 5)

Dups: 0




## Find Second Lowest Cost Silver Plan by Rate Area

In [313]:
df_plan = pd.read_csv('./plans.csv')
df_plan.head()

,plan_id,state,metal_level,rate,rate_area
0,74449NR9870320,GA,Silver,298.62,7
1,26325VH2723968,FL,Silver,421.43,60
2,09846WB8636633,IL,Gold,361.69,5
3,78590JQ3204809,SC,Gold,379.97,2
4,21163YD2193896,AR,Gold,359.75,2


In [314]:
df_plan.query('metal_level == "Silver"').sort_values(['rate_area', 'rate'])[['rate', 'rate_area']].head(10)

,rate,rate_area
17361,163.08,1
779,166.13,1
3639,166.13,1
6266,167.07,1
16140,167.07,1
12669,167.18,1
9592,168.24,1
14907,168.24,1
12754,168.48,1
18230,168.48,1


In [315]:
rate_areas = df_plan['rate_area'].unique()
print(rate_areas)

[ 7 60  5  2 11 18  1 23  9  3 17  4 14 15  8 13  6 25 19 51 47 12 53 22 10
 40 43 26 42 37 50 32 29 48 35 34 33 44 16 20 52 21 36 27 30 46 24 64 58 65
 31 38 57 49 39 67 28 62 45 41 61 56 63 55 54 59 66]


In [316]:
columns = ['rate_area', 'rate']
df_area_rates = pd.DataFrame(columns=columns)
df_area_rates

,rate_area,rate


In [317]:
i = 0
for area in rate_areas:
    rates_by_area = df_plan.query('rate_area == @area').sort_values('rate')['rate'].unique()
    second_lowest = rates_by_area[1]
    data = ({'rate_area': area, 'rate': second_lowest})
    df_area_rates.loc[i] = pd.Series(data)
    i += 1

In [318]:
df_area_rates.sort_values('rate_area').head(10)

,rate_area,rate
6,1.0,129.28
3,2.0,141.92
9,3.0,139.91
11,4.0,123.38
2,5.0,144.50
16,6.0,122.42
0,7.0,135.64
14,8.0,158.12
8,9.0,142.29
24,10.0,158.12


## Assign Rates to Zipcodes

In [319]:
df_slcsp = pd.read_csv('./slcsp.csv', dtype={'zipcode': str})
df_slcsp.head()

,zipcode,rate
0,64148,139.91
1,67118,122.42
2,40813,158.12
3,18229,139.91
4,51012,139.91


In [320]:
df_zip = pd.read_csv('./zips.csv', dtype={'zipcode': str})
df_zip.drop_duplicates(['zipcode', 'rate_area'], inplace=True)

In [321]:
df_zip_areas = df_slcsp.merge(df_zip[['zipcode', 'rate_area']], on='zipcode', how='inner')

In [322]:
df_zip_areas['dups'] = new_df.duplicated('zipcode')
df_zip_areas.drop(['rate'], axis=1, inplace=True)

In [323]:
df_zip_rates = df_zip_areas.merge(df_area_rates, on='rate_area', how='left')

## Find Ambiguous Zipcodes

In [324]:
ambig_zips = df_zip_rates.query('dups == True')['zipcode']

In [325]:
ambig_zips

7     54923
17    46706
19    14846
21    48872
22    48872
24    43343
31    48435
38    56097
46    63359
47    63359
52    48418
61    47452
62    47452
Name: zipcode, dtype: object

In [326]:
df_zip_rates.drop_duplicates(['zipcode'], inplace=True)

In [327]:
for ambig_zip in ambig_zips:
    index = df_zip_rates.index[df_zip_rates['zipcode'] == ambig_zip]
    df_zip_rates.loc[index, 'rate'] = None


## Clean and Output Data

In [328]:
df_zip_rates.drop(['rate_area', 'dups'], axis=1, inplace=True)

In [329]:
df_zip_rates.to_csv('./slcsp.csv', header=['zipcode', 'rate'], index=False)